In [1]:
from imprint.nb_util import setup_nb
setup_nb()

[worker_id=None] 2023-03-09 16:44:26,555 - imprint - DEBUG 
Enabling 64-bit floats in JAX.


In [24]:

import numpy as np
import imprint as ip
from imprint.models.ztest import ZTest1D
import confirm.adagrid as ada
d = 2
g = ip.cartesian_grid(
    theta_min=np.full(d, -1),
    theta_max=np.full(d, 0),
    null_hypos=[ip.hypo("theta0 > theta1")],
)
db = ada.ada_validate(ZTest1D, g=g, lam=-1.96, prod=False, tile_batch_size=1)

[worker_id=None] 2023-03-09 16:49:11,301 - imprint.grid - DEBUG 
_gen_short_uuids(n=4, worker_id=1, t=1678398567, n_bits=18, worker_bits=18) = [4658206836435189760 4658206836435189761 4658206836435189762, ...]:
[worker_id=None] 2023-03-09 16:49:11,307 - imprint.grid - DEBUG 
_gen_short_uuids(n=4, worker_id=1, t=1678398568, n_bits=18, worker_bits=18) = [4658206905154666496 4658206905154666497 4658206905154666498, ...]:
[worker_id=1] 2023-03-09 16:49:11,397 - confirm.adagrid.init - DEBUG 
Initialized database with 3 tiles and 1 null hypos. The tiles are split between 1 zones with packet_size=1024.
[worker_id=1] 2023-03-09 16:49:11,403 - confirm.adagrid.backend - DEBUG 
init took 0.08212828636169434
[worker_id=1] 2023-03-09 16:49:11,404 - confirm.adagrid.backend - DEBUG 
setup took 2.9087066650390625e-05
[worker_id=1] 2023-03-09 16:49:11,404 - confirm.adagrid.step - DEBUG 
waiting for work: (zone_id=0, step_id=0, packet_id=0)
[worker_id=1] 2023-03-09 16:49:11,407 - confirm.adagrid.step - 

In [3]:
import os
import coiled
import imprint 
import confirm
from distributed.diagnostics.plugin import UploadDirectory

coiled.create_software_environment(name="confirm-coiled", conda="env-coiled.yml")
cluster = coiled.Cluster(
    name="confirm-coiled",
    software="confirm-coiled",
    n_workers=1,
    worker_vm_types=["g4dn.xlarge"],
    worker_gpu=1,
    compute_purchase_option='spot_with_fallback',
    shutdown_on_close=False
)
client = cluster.get_client()

Output()

/Users/tbent/.mambaforge/envs/confirm/lib/python3.10/site-packages/distributed/client.py:1361: VersionMismatchWarning: Mismatched versions found

+---------+--------+-----------+---------+
| Package | Client | Scheduler | Workers |
+---------+--------+-----------+---------+
| lz4     | 4.3.2  | None      | None    |
+---------+--------+-----------+---------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


In [4]:
import dask
@dask.delayed
def check_nvidia_jax():
    import jax
    import subprocess
    result = subprocess.run(['nvidia-smi'], stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    nvidia_smi = result.stdout.decode('ascii')
    jax_platform = jax.lib.xla_bridge.get_backend().platform
    key = jax.random.PRNGKey(0)
    means = jax.random.uniform(key, (10000, 2)).mean(axis=0)
    return nvidia_smi, jax_platform, means

nvidia_smi, jax_platform, means = check_nvidia_jax().compute()
print(nvidia_smi)
jax_platform, means

Thu Mar  9 21:44:41 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                        On | 00000000:00:1E.0 Off |                    0 |
| N/A   38C    P0               26W /  70W|  11625MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

('gpu', Array([0.50014436, 0.49666256], dtype=float64))

In [5]:
def upload_pkg(client, module, restart=False):
    from pathlib import Path

    dir = Path(module.__file__).parent.parent
    skip = [p for p in os.listdir(dir) if p != module.__name__]
    return client.register_worker_plugin(
        UploadDirectory(dir, skip_words=skip, restart=restart, update_path=True),
        nanny=True,
    )

upload_pkg(client, confirm)
upload_pkg(client, imprint, restart=True)

{'tls://10.0.55.123:37075': {'status': 'OK'}}

In [6]:
import dask

@dask.delayed
def run_ada():
    import synchronicity
    import numpy as np
    import imprint as ip
    from imprint.models.ztest import ZTest1D
    import confirm.adagrid as ada
    ip.package_settings()
    d = 2
    g = ip.cartesian_grid(
        theta_min=np.full(d, -1),
        theta_max=np.full(d, 0),
        null_hypos=[ip.hypo("theta0 > theta1")],
    )
    db = ada.ada_validate(
        ZTest1D,
        g=g,
        lam=-1.96,
        prod=False
    )
    return ip.Grid(db.get_results(), 1).prune_inactive().n_tiles

NN = run_ada().compute()
NN

355

In [17]:
import time
import jax
import numpy as np
import imprint as ip
from confirm.adagrid.validate import AdaValidate
from confirm.models.wd41 import WD41
from distributed import get_worker
    
def setup_worker_algo(worker_args):
    worker = get_worker()
    (model_type, model_args, model_kwargs, algo_type, cfg) = worker_args
    hash_args = hash((
        model_type.__name__,
        model_args,
        tuple(model_kwargs.items()),
        algo_type.__name__,
        tuple(cfg.items()),
    ))
    has_hash = hasattr(worker, 'algo_hash')
    has_algo = hasattr(worker, 'algo')
    if not (has_algo and has_hash) or (has_hash and hash_args != worker.algo_hash):
        model = model_type(*model_args, **model_kwargs)
        cfg['worker_id'] = 2
        worker.algo = algo_type(model, None, None, cfg, None) 
        worker.algo_hash = hash_args
    async def async_process_tiles(tiles_df):
        return await worker.algo.process_tiles(tiles_df = tiles_df)
    import synchronicity
    synchronizer = synchronicity.Synchronizer()
    process_tiles = synchronizer.create(async_process_tiles)[synchronicity.Interface.BLOCKING]
    return process_tiles

@dask.delayed
def dask_process_packet(worker_args, packet_df):
    process_tiles = setup_worker_algo(worker_args)
    jax_platform = jax.lib.xla_bridge.get_backend().platform
    assert jax_platform == 'gpu'
    out = process_tiles(packet_df)
    return out

g = ip.cartesian_grid(
    theta_min=np.full(4, -1),
    theta_max=np.full(4, 0),
    n=[1, 10, 10, 10],
    null_hypos=[ip.hypo("theta0 > theta1")],
)
K = 2**13
g.df['K'] = K
cfg = dict(
    init_K = K,
    n_K_double = 3,
    tile_batch_size = 64,
    lam = -1.96, 
    delta = 0.01,
    global_target = 0.001,
    max_target = 0.01,
)

worker_args = (WD41, (0, K), dict(ignore_intersection=True), AdaValidate, cfg)
worker_args_future = client.scatter(worker_args, broadcast=True)
start = time.time()
out = dask_process_packet(worker_args_future, g.df).compute()
runtime1 = time.time() - start
start = time.time()
out = dask_process_packet(worker_args_future, g.df).compute()
runtime2 = time.time() - start
print(runtime1, runtime2)

[worker_id=None] 2023-03-09 16:48:21,092 - imprint.grid - DEBUG 
_gen_short_uuids(n=1000, worker_id=1, t=1678398506, n_bits=18, worker_bits=18) = [4658202644547108864 4658202644547108865 4658202644547108866, ...]:
[worker_id=None] 2023-03-09 16:48:21,095 - imprint.planar_null - DEBUG 
HyperPlane has dimension 2 but grid has dimension 4. Padding with zeros.
[worker_id=None] 2023-03-09 16:48:21,097 - imprint.planar_null - DEBUG 
HyperPlane has dimension 2 but grid has dimension 4. Padding with zeros.
[worker_id=None] 2023-03-09 16:48:21,100 - imprint.planar_null - DEBUG 
HyperPlane has dimension 2 but grid has dimension 4. Padding with zeros.
[worker_id=None] 2023-03-09 16:48:21,113 - imprint.planar_null - DEBUG 
HyperPlane has dimension 2 but grid has dimension 4. Padding with zeros.
[worker_id=None] 2023-03-09 16:48:21,158 - imprint.grid - DEBUG 
_gen_short_uuids(n=2000, worker_id=1, t=1678398507, n_bits=18, worker_bits=18) = [4658202713266585600 4658202713266585601 4658202713266585602

In [21]:
class CoiledAlgo:
    def __init__(self, client, worker_args_future, wrapped):
        super().__init__()
        self.client = client
        self.wrapped = wrapped
        self.worker_args_future = worker_args_future

        self.null_hypos = wrapped.null_hypos
        self.cfg = wrapped.cfg
        self.db = wrapped.db
        self.callback = wrapped.callback
        self.Ks = wrapped.Ks
        self.max_K = wrapped.max_K
        self.driver = wrapped.driver
        
    def get_orderer(self):
        return self.wrapped.get_orderer()
        
    async def process_tiles(self, *, tiles_df):
        return dask_process_packet(self.worker_args_future, tiles_df).compute()
        
    async def convergence_criterion(self, zone_id, report):
        return await self.wrapped.convergence_criterion(zone_id, report)

    async def select_tiles(self, zone_id, new_step_id, report, convergence_task):
        return await self.wrapped.select_tiles(zone_id, new_step_id, report, convergence_task)

In [41]:
import contextlib
from confirm.adagrid.backend import LocalBackend
class CoiledBackend(LocalBackend):
    def __init__(self, client, n_zones=1, coordinate_every=5):
        super().__init__(n_zones, coordinate_every)
        self.client = client

    @contextlib.asynccontextmanager
    async def setup(self, algo_type, algo, kwargs):
        algo_entries = [
            'init_K',
            'n_K_double',
            'tile_batch_size',
            'lam',
            'delta',
            'worker_id',
            'global_target',
            'max_target',
            'bootstrap_seed',
            'nB',
            'alpha',
            'calibration_min_idx'
        ]
        filtered_cfg = {k: v for k, v in algo.cfg.items() if k in algo_entries}
        worker_args = (type(algo.driver.model), (algo.cfg['model_seed'], algo.max_K), algo.cfg['model_kwargs'], algo_type, filtered_cfg)
        worker_args_future = client.scatter(worker_args, broadcast=True)
        self.algo = CoiledAlgo(self.client, worker_args_future, algo)
        yield

In [42]:
import numpy as np
import imprint as ip
from imprint.models.ztest import ZTest1D
import confirm.adagrid as ada

d = 2
g = ip.cartesian_grid(
    theta_min=np.full(d, -1),
    theta_max=np.full(d, 0),
    null_hypos=[ip.hypo("theta0 > theta1")],
)
db = ada.ada_calibrate(ZTest1D, g=g, alpha=0.025, prod=False, backend=CoiledBackend(client))

[worker_id=None] 2023-03-09 16:56:17,154 - imprint.grid - DEBUG 
_gen_short_uuids(n=4, worker_id=1, t=1678398977, n_bits=18, worker_bits=18) = [4658235011420651520 4658235011420651521 4658235011420651522, ...]:
[worker_id=None] 2023-03-09 16:56:17,159 - imprint.grid - DEBUG 
_gen_short_uuids(n=4, worker_id=1, t=1678398978, n_bits=18, worker_bits=18) = [4658235080140128256 4658235080140128257 4658235080140128258, ...]:
[worker_id=1] 2023-03-09 16:56:17,242 - confirm.adagrid.init - DEBUG 
Initialized database with 3 tiles and 1 null hypos. The tiles are split between 1 zones with packet_size=1024.
[worker_id=1] 2023-03-09 16:56:18,563 - confirm.adagrid.backend - DEBUG 
init took 1.3963210582733154
[worker_id=1] 2023-03-09 16:56:18,582 - confirm.adagrid.backend - DEBUG 
setup took 0.01878976821899414
[worker_id=1] 2023-03-09 16:56:18,583 - confirm.adagrid.step - DEBUG 
waiting for work: (zone_id=0, step_id=0, packet_id=0)
[worker_id=1] 2023-03-09 16:56:18,586 - confirm.adagrid.step - DEBU

In [44]:
import confirm.models.wd41 as wd41
model = wd41.WD41(0, 1, ignore_intersection=True)
grid = ip.cartesian_grid(
    [-2.5, -2.5, -2.5, -2.5],
    [1.0, 1.0, 1.0, 1.0],
    n=[10, 10, 10, 10],
    null_hypos=model.null_hypos,
)
db = ada.DuckDBTiles.connect('./wd41-3.db')
ada.ada_calibrate(
    wd41.WD41,
    g=grid,
    db=db,
    alpha=0.025,
    bias_target=0.0025,
    grid_target=0.0025,
    std_target=0.005,
    n_K_double=6,
    calibration_min_idx=100,
    step_size=2**18,
    packet_size=2**15,
    model_kwargs={"ignore_intersection": True},
    backend=CoiledBackend(client)
)

[worker_id=None] 2023-03-09 16:57:13,565 - imprint.grid - DEBUG 
_gen_short_uuids(n=10000, worker_id=1, t=1678399033, n_bits=18, worker_bits=18) = [4658238859711348736 4658238859711348737 4658238859711348738, ...]:
[worker_id=None] 2023-03-09 16:57:13,570 - imprint.planar_null - DEBUG 
HyperPlane has dimension 2 but grid has dimension 4. Padding with zeros.
[worker_id=None] 2023-03-09 16:57:13,576 - imprint.planar_null - DEBUG 
HyperPlane has dimension 2 but grid has dimension 4. Padding with zeros.
[worker_id=None] 2023-03-09 16:57:13,581 - imprint.planar_null - DEBUG 
HyperPlane has dimension 2 but grid has dimension 4. Padding with zeros.
[worker_id=None] 2023-03-09 16:57:13,624 - imprint.planar_null - DEBUG 
HyperPlane has dimension 2 but grid has dimension 4. Padding with zeros.
[worker_id=None] 2023-03-09 16:57:13,651 - imprint.grid - DEBUG 
_gen_short_uuids(n=2000, worker_id=1, t=1678399034, n_bits=18, worker_bits=18) = [4658238928430825472 4658238928430825473 465823892843082547

In [58]:
client.close()
cluster.close()